In [ ]:
import requests
import base64
import time
import os
import gzip
import pandas as pd
import zipfile

Connect and download from Amplitude

In [ ]:
def download_from_amplitude(api_key, secret_key, date_start, date_end, file_name='amplitude_data',url="https://amplitude.com/api/2/export"):
    credentials = base64.b64encode(f"{api_key}:{secret_key}".encode())
    headers = {"Authorization": f"Basic {credentials.decode()}"}
    params = {"start": date_start, "end": date_end}
    response = requests.get(url, params=params, headers=headers, stream=True)
    if response.status_code == 200:
        print('Connected to Amplitude server')
        # Simulate downloading progress
        print('Downloading data... [          ]', end='', flush=True)
        for i in range(10):
            time.sleep(0.5)  # Simulate downloading with a delay
            print('\b#', end='', flush=True)
        with open(f"{file_name}_{date_start}_{date_end}.zip", "wb") as f:
            f.write(response.content)
        print('\nAmplitude data downloaded and saved to zip file:', f"{file_name}_{date_start}_{date_end}.zip")
        return 'Download completed'
    else:
        print('Error:', response.text)
        return 'Download failed'

In [ ]:
# Example usage:
api_key = "99d374f67220dcf1643e35409ff9b91e"
secret_key = "33dbf6d46e7c69ddb1f26a76789430c2"
start_date = "20240315"
end_date = "20240317"

result = download_from_amplitude(api_key, secret_key, start_date, end_date)
print(result)  # Print the status message returned by the function

Extract zip file and convert JSON to the Pandas Dataframe

In [ ]:
# Extract zip file to gz file in folder amplitude_data
zip_file = f'amplitude_data_{start_date}_{end_date}.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
   zip_ref.extractall('amplitude_data')

In [ ]:
# Open unzipped folder. Change YOUR_FOLDER_NAME to the name in your path. For example ‘amplitude_data/430255’ folder_name = '430255’
folder_name = input('input_foldername: ')

path = f'amplitude_data/{folder_name}' 
files = os.listdir(path)

# Create DataFrame
df = pd.DataFrame()
# Add all JSON files from ‘path’ to the Dataframe 
for file in files:
    df_i = pd.read_json(f'{path}/{file}',lines=True,compression='gzip')
    df = pd.concat([df, df_i], axis=0)

Save to csv file

In [ ]:
# Save to CSV
df.to_csv(f'amplitude_data_{start_date}_{end_date}.csv', index=False)